# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Shi  ->  J. Shi  |  ['J. Shi']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
C. Gieser  ->  C. Gieser  |  ['C. Gieser']
D. Semenov  ->  D. Semenov  |  ['D. Semenov']
M. Wells  ->  M. Wells  |  ['M. Wells']


Arxiv has 86 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2512.09020
extracting tarball to tmp_2512.09020... done.
Retrieving document from  https://arxiv.org/e-print/2512.09339


extracting tarball to tmp_2512.09339... done.
Retrieving document from  https://arxiv.org/e-print/2512.09359


extracting tarball to tmp_2512.09359...

 done.
Retrieving document from  https://arxiv.org/e-print/2512.09519


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2512.09359/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2512.09519...

 done.


Issues with the citations
list index out of range


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.09519-b31b1b.svg)](https://arxiv.org/abs/2512.09519) | **CASCADE: Filamentary accretion flows in Cygnus X DR20**  |
|| M. Sawczuck, et al. -- incl., <mark>H. Beuther</mark>, <mark>C. Gieser</mark>, <mark>D. Semenov</mark>, <mark>M. Wells</mark> |
|*Appeared on*| *2025-12-11*|
|*Comments*| *16 pages, 14 figures, accepted for A&A*|
|**Abstract**|            Aims. We investigate the role of filaments in high-mass star formation, whether gas flows from large to small scales along them, and what their properties might reveal about the region they are found in. Methods. The Max Planck IRAM Observatory Program (MIOP), the Cygnus Allscale Survey of Chemistry and Dynamical Environments (CASCADE), includes high spatial resolution (~3'') data of HCO+(1-0) and H13CO+(1-0) emission in the star-forming DR20 region in the Cygnus X complex. In this data we identify filaments with the structure identification algorithm DisPerSE. We further analyze these filaments using Gaussian fits to the spectra to determine the line peak velocity and full width half maximum along them. The Python package FilChaP was used to determine filament widths. Results. We find projected velocity gradients inside several filaments between 0.4 to 2.4km/s over projected length-scales of 0.1pc toward star-forming cores. This can be interpreted as a sign of gas flowing along the filaments toward the cores. The filament width distributions exhibit median values between 0.06 and 0.14pc depending on the core, the tracer, and the method. Standard deviations are approximately 0.02 to 0.06pc. These values are roughly in agreement with the filament width of 0.1pc typically found in nearby low-mass star-forming regions. Conclusions. This first analysis of filamentary properties within the Cygnus X CASCADE program reveals potential signatures of gas flows along filaments onto star-forming cores. Furthermore, the characteristics of the filaments in this high-mass star-forming region can be compared to those of filaments in low-mass star-forming regions typically studied before. Extending such studies to the entire CASCADE survey will enhance our knowledge of high-mass filament properties on solid statistical grounds.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.09020-b31b1b.svg)](https://arxiv.org/abs/2512.09020) | **University of Hawaii 88-inch Telescope Observations of the Interstellar Comet 3I/ATLAS: Spectrophotometric Blue-Sensitive Spectral Time Series Spanning Two Months from Discovery**  |
|| W. B. Hoogendam, et al. -- incl., <mark>J. Shi</mark> |
|*Appeared on*| *2025-12-11*|
|*Comments*| *Comments welcome! 11 pages, 5 figures, to be submitted to the Open Journal of Astrophysics*|
|**Abstract**|            Interstellar objects are the ejected building blocks of other solar systems. As such, they enable the acquisition of otherwise inaccessible information about nascent extrasolar systems. The discovery of the third interstellar object, 3I/ATLAS, provides an opportunity to explore the properties of a small body from another solar system and to compare it to the small bodies in our own. To that end, we present spectrophotometric observations of 3I/ATLAS taken using the SuperNova Integral Field Spectrograph on the University of Hawaii 2.2-m telescope. Our data includes the earliest $\lambda\leq3800$ A spectrum of 3I/ATLAS, obtained $\sim$12.5 hours after the discovery announcement. Later spectra confirm previously reported cometary activity, including Ni and CN emission. The data show wavelength-varying spectral slopes ($S\approx($0\%-29\%)/1000 A, depending on wavelength range) throughout the pre-perihelion ($r_h=4.4$-$2.5$ au) approach of 3I/ATLAS. We perform synthetic photometry on our spectra and find 3I/ATLAS shows mostly stable color evolution over the period of our observations, with $g-r$ colors ranging from $\sim$0.69-0.75 mag, $r-i$ colors ranging from $\sim$0.26-0.30 mag, and $c-o$ colors ranging from $\sim$0.50-0.55 mag. Ongoing post-perihelion observations of 3I/ATLAS will provide further insight into its potentially extreme composition.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.09339-b31b1b.svg)](https://arxiv.org/abs/2512.09339) | **Discovery of the redback millisecond pulsar PSR J1728-4608 with ASKAP**  |
|| F. Petrou, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-12-11*|
|*Comments*| *Accepted in PASA, manuscript ID PAS-2025-0177.R3*|
|**Abstract**|            We present the discovery of PSR J1728-4608, a new redback spider pulsar identified in images from the Australian SKA Pathfinder telescope. PSR J1728-4608 is a millisecond pulsar with a spin period of 2.86 ms, in a 5.05 hr orbit with a companion star. The pulsar exhibits a radio spectrum of the form $S_\nu \propto \nu^\alpha$, with a measured spectral index of $\alpha = -1.8(3)$. It is eclipsed for 42% of its orbit at 888 MHz, and multi--frequency image--domain observations show that the egress duration scales with frequency as a power law with index $n = -1.74$, where longer duration eclipses are seen at lower frequencies. An optical counterpart is detected in archival Gaia data within $0.5''$ of the radio position. It has a mean G-band magnitude of 18.8 mag and its light curve displays characteristics consistent with a combination of ellipsoidal modulation and irradiation effects. We also report the nearest Fermi $\gamma$-ray source, located 2$'$ away from our source, as a possible association. A radio timing study constrains the intrinsic and orbital properties of the system, revealing orbital period variations that we attribute to changes in the gravitational quadrupole moment of the companion star. At the eclipse boundary, we measure a maximum dispersion measure excess of $2.0 \pm 1.2 \ \mathrm{pc\ cm^{-3}}$, corresponding to an electron column density of $5.9 \pm 3.6 \times10^{18} \ \mathrm{cm^{-2}}$. Modelling of the eclipse mechanism suggests that synchrotron absorption is the dominant cause of the eclipses observed at radio wavelengths. The discovery and characterisation of systems like \psr\ provide valuable insights into pulsar recycling, binary evolution, the nature of companion-driven eclipses, and the interplay between compact objects and their plasma environments.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.09359-b31b1b.svg)](https://arxiv.org/abs/2512.09359) | **CWTHF: Subhalo Identification with Continuous Wavelet Transform**  |
|| M. Li, <mark>Y. Wang</mark>, P. He |
|*Appeared on*| *2025-12-11*|
|*Comments*| *16pages, 12 figures*|
|**Abstract**|            With advances in cosmology and computer science, cosmological simulations now resolve structures in increasingly fine detail. As key tracers of hierarchical structure formation, subhalos are among the most important objects within these simulations. In our previous work, we established that the continuous wavelet transform (CWT) can effectively extract clustering information and serve as a robust halo finder. Here, we extend the CWT framework to subhalo identification by adapting the CWTHF (Continuous Wavelet Transform Halo Finder) code. This extension extends the unbinding procedure, which enables the reliable identification of gravitationally bound substructures. The algorithm identifies density peaks within known halos or subhalos and segments the surrounding volume accordingly. Once a new subhalo is registered, its position is recorded to prevent duplicate detection. We validate our approach using the TNG50-2 and TNG100-1 simulations, as well as a single Friends-of-Friends (FOF) halo, by comparing the resulting CWT catalog against the reference SUBFIND catalog. Because the method inherits the original computational framework, our subhalo finder maintains a favorable linear time complexity of $\mathcal{O}(N)$.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2512.09519/./Images/Along_A5_4plots_samey_yellow.png', 'tmp_2512.09519/./Images/Along_B3_4plots_samey_yellow.png', 'tmp_2512.09519/./Images/Along_D2_4plots_samey_yellow.png', 'tmp_2512.09519/./Images/Along_Arem2_4plots_samey_newerr.png', 'tmp_2512.09519/./Images/HCO+_NOEMA_fil_newcol_6sig_NDSKL_noAF_beam.png', 'tmp_2512.09519/./Images/H13CO+_NOEMA_fil_newcol_6sig_NDSKL_AF_beam.png', 'tmp_2512.09519/./Images/HCO+_narrow_fil_4sig_NDskl_newcolors_beam_cont.png', 'tmp_2512.09519/./Images/H13CO+_narrow_fil_4sig_NDskl_newcolors_beam_cont.png', 'tmp_2512.09519/./Images/Filchap_all_gaussnew.png', 'tmp_2512.09519/./Images/Filchap_AB_gaussnew.png', 'tmp_2512.09519/./Images/Filchap_CF_gaussnew.png', 'tmp_2512.09519/./Images/Filchap_all_plum2new.png', 'tmp_2512.09519/./Images/Filchap_AB_plum2new.png', 'tmp_2512.09519/./Images/Filchap_CF_plum2new.png', 'tmp_2512.09519/./Images/Filchap_all_plum4new.png', 'tmp_2512.09519/./Images/Filchap_AB_plum4new.png', 'tmp_2512.09519/./Images/

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\arraystretch}{1.5}$
$\newcommand\natexlab{#1}$</div>



<div id="title">

# CASCADE: Filamentary accretion flows in Cygnus X DR20

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.09519-b31b1b.svg)](https://arxiv.org/abs/2512.09519)<mark>Appeared on: 2025-12-11</mark> -  _16 pages, 14 figures, accepted for A&A_

</div>
<div id="authors">

M. Sawczuck, et al. -- incl., <mark>H. Beuther</mark>, <mark>C. Gieser</mark>, <mark>D. Semenov</mark>, <mark>M. Wells</mark>

</div>
<div id="abstract">

**Abstract:** Filamentary gas flows are an important process for funneling gas from cloud scales onto star-forming cores. We investigate the role of filaments in high-mass star formation, whether gas flows from large to small scales along them, and what their properties might reveal about the region they are found in. The Max Planck IRAM Observatory Program (MIOP), the Cygnus Allscale Survey of Chemistry and Dynamical Environments (CASCADE), includes high spatial resolution ( $\sim$ 3 $"$ ) data of $\HCO$ $(1-0)$ and $\HttCO$ $(1-0)$ emission in the star-forming DR20 region in the Cygnus X complex. In this data we identify filaments with the structure identification algorithm DisPerSE. We further analyze these filaments using Gaussian fits to the spectra to determine the line peak velocity and full width half maximum along them. The Python package FilChaP was used to determine filament widths. We find projected velocity gradients inside several filaments between 0.4 to 2.4 $\kms$ over projected length-scales of 0.1 pc toward star-forming cores. This can be interpreted as a sign of gas flowing along the filaments toward the cores. The filament width distributions exhibit median values between 0.06 and 0.14 pc depending on the core, the tracer, and the method. Standard deviations are approximately 0.02 to 0.06 pc. These values are roughly in agreement with the filament width of 0.1 pc typically found in nearby low-mass star-forming regions. This first analysis of filamentary properties within the Cygnus X CASCADE program reveals potential signatures of gas flows along filaments onto star-forming cores. Furthermore, the characteristics of the filaments in this high-mass star-forming region can be compared to those of filaments in low-mass star-forming regions typically studied before. Extending such studies to the entire CASCADE survey will enhance our knowledge of high-mass filament properties on solid statistical grounds.

</div>

<div id="div_fig1">

<img src="tmp_2512.09519/./Images/Along_A5_4plots_samey_yellow.png" alt="Fig8.1" width="25%"/><img src="tmp_2512.09519/./Images/Along_B3_4plots_samey_yellow.png" alt="Fig8.2" width="25%"/><img src="tmp_2512.09519/./Images/Along_D2_4plots_samey_yellow.png" alt="Fig8.3" width="25%"/><img src="tmp_2512.09519/./Images/Along_Arem2_4plots_samey_newerr.png" alt="Fig8.4" width="25%"/>

**Figure 8. -** FWHM and peak positions of the Gaussian fits to the spectra within filaments connected to cores A, B, D, and a remote area in \HCO._From top to bottom_: Cores A, B, D, and a remote area close to core A, respectively (A5, B3, D2, A$_{\text{rem}}$2). _Images_: \HCO$(1-0)$ zeroth moment map (see top left of Fig. \ref{plot_MIOP_filaments_0mom}). The white dot within the core indicates the coordinates of the continuum source, while the yellow dot indicates the filament “origin,” as described in the main text. The filament corresponding to the plots is shown in its velocity colors. Other analyzed filaments are shown in green, their plots can be found in supplementary material at Zenodo (Figs. B.1 and B.16). _Plots_: Gaussian fitted HCO$^+$ FWHM (left) and peak positions (right) of the spectra within the corresponding filament, plotted over the distance from the core. The yellow dot in the image corresponds to a distance of zero. If there is a significant amount of data points ($\geq$ 10) for a second spectral Gaussian peak, the positions of the higher velocity peaks (top) and the lower velocity peaks (bottom) are shown in separate plots. The peaks that do not clearly belong to either of the two components are shown in gray. The other data points are colored by their fitted peak positions. (*plot_along_chosen_HCO*)

</div>
<div id="div_fig2">

<img src="tmp_2512.09519/./Images/HCO+_NOEMA_fil_newcol_6sig_NDSKL_noAF_beam.png" alt="Fig6.1" width="25%"/><img src="tmp_2512.09519/./Images/H13CO+_NOEMA_fil_newcol_6sig_NDSKL_AF_beam.png" alt="Fig6.2" width="25%"/><img src="tmp_2512.09519/./Images/HCO+_narrow_fil_4sig_NDskl_newcolors_beam_cont.png" alt="Fig6.3" width="25%"/><img src="tmp_2512.09519/./Images/H13CO+_narrow_fil_4sig_NDskl_newcolors_beam_cont.png" alt="Fig6.4" width="25%"/>

**Figure 6. -** Identified filaments in combined and 30 m \HCO  and \HttCO.Filaments identified with DisPerSE in all velocity channels (from -8.4 km s$^{-1}$ to 6 km s$^{-1}$) overlaid on the respective zeroth moment maps, i.e., the intensity integrated over all velocity channels, of all CASCADE data types. _Top_: Combined \HCO$(1-0)$(left) and \HttCO$(1-0)$(right). _Bottom_: Single-dish \HCO$(1-0)$(left) and \HttCO$(1-0)$(right). The intensity maxima of the 3.6 mm continuum emission are labeled A-F in the upper right panel. The beam is shown in the bottom-right of each panel. (*plot_MIOP_filaments_0mom*)

</div>
<div id="div_fig3">

<img src="tmp_2512.09519/./Images/Filchap_all_gaussnew.png" alt="Fig12.1" width="8%"/><img src="tmp_2512.09519/./Images/Filchap_AB_gaussnew.png" alt="Fig12.2" width="8%"/><img src="tmp_2512.09519/./Images/Filchap_CF_gaussnew.png" alt="Fig12.3" width="8%"/><img src="tmp_2512.09519/./Images/Filchap_all_plum2new.png" alt="Fig12.4" width="8%"/><img src="tmp_2512.09519/./Images/Filchap_AB_plum2new.png" alt="Fig12.5" width="8%"/><img src="tmp_2512.09519/./Images/Filchap_CF_plum2new.png" alt="Fig12.6" width="8%"/><img src="tmp_2512.09519/./Images/Filchap_all_plum4new.png" alt="Fig12.7" width="8%"/><img src="tmp_2512.09519/./Images/Filchap_AB_plum4new.png" alt="Fig12.8" width="8%"/><img src="tmp_2512.09519/./Images/Filchap_CF_plum4new.png" alt="Fig12.9" width="8%"/><img src="tmp_2512.09519/./Images/Filchap_all_momnew.png" alt="Fig12.10" width="8%"/><img src="tmp_2512.09519/./Images/Filchap_AB_momnew.png" alt="Fig12.11" width="8%"/><img src="tmp_2512.09519/./Images/Filchap_CF_momnew.png" alt="Fig12.12" width="8%"/>

**Figure 12. -** Filament widths in \HCO of all analyzed filaments, and separated into younger and evolved cores, for all methods.All calculated de-convolved filament widths resulting from the different fits performed with FilChaP on the combined \HCO$(1-0)$ data. _Left column_: All regions including cores A to F and the remote filaments close to A and B. _Middle column_: Only filaments directly connected to the evolved regions A and B. _Right column_: Only filaments directly connected to the younger regions C, D, E, and F. _Rows_: Different fits, i.e., Gaussian, Plummer $p$ = 2 and $p$ = 4, and second moment, from top to bottom. (*plot_filchap_AB_CF*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.09519"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

139  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
